In [1]:
import pandas as pd
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

from data_generator import DFIterator
from model_builder import ModelBuilder

In [2]:
COLUMNS = ['file', 'X', 'Y', 'Z', 'W', 'P', 'Q', 'R']
IMG_COLUMN = 'file'
POSITION_COLUMNS = ['X', 'Y', 'Z']
ORIENTATION_COLUMNS = ['W', 'P', 'Q', 'R']

DIRECTORY = 'KingsCollege/'
TRAIN_PATH = 'KingsCollege/dataset_train.txt'
TEST_PATH = 'KingsCollege/dataset_test.txt'
SKIPROWS = 2
COLUMN_SEP = ' '
HEADER = None

IMG_SIZE = (256, 256)
IMG_CROP_SIZE = (224, 224)
POSE_REGRESSOR_SIZE = 1024
BETA = 500
BATCH_SIZE = 32
SHUFFLE = True

In [3]:
train_data = pd.read_csv(TRAIN_PATH, skiprows=SKIPROWS, sep=COLUMN_SEP, header=HEADER).rename(
        columns={i: c for i, c in enumerate(COLUMNS)})
test_data = pd.read_csv(TEST_PATH, skiprows=SKIPROWS, sep=COLUMN_SEP, header=HEADER).rename(
        columns={i: c for i, c in enumerate(COLUMNS)})

In [4]:
imagegen = ImageDataGenerator()
traingen = DFIterator(
    directory=DIRECTORY, dataframe=train_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

testgen = DFIterator(
    directory=DIRECTORY, dataframe=test_data, image_data_generator=imagegen,
    x_col=IMG_COLUMN, y_col=POSITION_COLUMNS+ORIENTATION_COLUMNS,
    target_size=IMG_SIZE, random_crop_shape=IMG_CROP_SIZE,
    class_mode='raw', batch_size=BATCH_SIZE, shuffle=SHUFFLE)

Found 1220 validated image filenames.
Found 343 validated image filenames.


In [5]:
model_builder = ModelBuilder()

In [6]:
model = model_builder.build_model(image_size=IMG_CROP_SIZE, pose_regressor_size=POSE_REGRESSOR_SIZE)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model_builder.compile_model(model, lr=0.0001, momentum=0.9, beta=200)

In [10]:
model.fit_generator(traingen, epochs=30, validation_data=testgen)

Epoch 1/30
39/39 [==============================] - 62s 2s/step - loss: 157.1351 - xyz_loss: 38.3652 - wpqr_loss: 0.3544 - wpqr_orientation_angle: 40.9925 - val_loss: 196.5531 - val_xyz_loss: 84.9109 - val_wpqr_loss: 0.3293 - val_wpqr_orientation_angle: 38.0172
Epoch 2/30
32/39 [=======================>......] - ETA: 5s - loss: 151.2675 - xyz_loss: 38.2828 - wpqr_loss: 0.3577 - wpqr_orientation_angle: 41.3864

KeyboardInterrupt: 